Part 1 and 2

In [1]:
import requests
import json

def getNews(source):
    url = 'https://newsapi.org/v2/top-headlines'
    parameters = {
    'sources': source,
    'apiKey' : 'cfc72f4f40754e25a303dbd907452397'
    }
    resp = requests.get(url,params=parameters)
    data = resp.json()
    url_list = []
    for i in data['articles']:
        url_list.append(i['url'])
    return(url_list)


def query_api(url):
    endpoint_watson = "https://gateway.watsonplatform.net/natural-language-understanding/api/v1/analyze"
    params = {
        'version': '2017-02-27',
    }
    headers = {
        'Content-Type': 'application/json',
    }
    options = {
        "url": url,
        "features": {
            "entities": {
                "sentiment" : True,
                "emotion" : False,
                "limit" : 20
            }
        }
    }
    username = "711b4792-170d-490f-ac0e-5d785a271868"
    password = "No7JUOE1UOYu"
    
    resp = requests.post(endpoint_watson,
                        data=json.dumps(options),
                        headers=headers,
                        params=params,
                        auth=(username, password)
                        )
    return resp.json()
    

In [2]:
getNews('Bloomberg')

['https://www.bloomberg.com/news/articles/2018-10-03/trump-mocks-kavanaugh-accusers-as-confirmation-hangs-in-balance',
 'https://www.bloomberg.com/news/articles/2018-10-03/trump-slams-nyt-article-on-family-s-tax-practices-as-hit-piece',
 'https://www.bloomberg.com/news/articles/2018-10-03/may-tells-u-k-conservatives-end-of-austerity-is-in-sight',
 'https://www.bloomberg.com/news/articles/2018-10-03/putin-tells-trump-to-blame-guy-in-the-mirror-for-high-oil-prices',
 'https://www.bloomberg.com/view/articles/2018-10-03/aston-martin-ipo-attracts-short-sellers',
 'https://www.bloomberg.com/news/articles/2018-10-03/saudis-raise-oil-output-to-near-record-amid-pressure-from-trump?utm_source%3Dgoogle%26utm_medium%3Dbd%26cmpId%3Dgoogle',
 'https://www.bloomberg.com/news/articles/2018-10-03/fed-s-evans-says-he-s-comfortable-with-prospect-of-december-hike',
 'https://www.bloomberg.com/news/articles/2018-10-03/malaysia-to-charge-ex-premier-najib-s-wife-with-money-laundering',
 'https://www.bloomber

In [3]:
url = 'https://apnews.com/c42aa22bb2f04de79d192b7cbd0c3fe9'
data_response = query_api(url)
data_response

{'entities': [{'count': 28,
   'disambiguation': {'dbpedia_resource': 'http://dbpedia.org/resource/Bill_Cosby',
    'name': 'Bill Cosby',
    'subtype': ['Actor',
     'MusicalArtist',
     'AwardNominee',
     'AwardWinner',
     'BroadcastArtist',
     'FilmActor',
     'FilmDirector',
     'FilmMusicContributor',
     'FilmProducer',
     'FootballPlayer',
     'HallOfFameInductee',
     'TVActor',
     'TVDirector',
     'TVPersonality',
     'TVProducer',
     'FilmWriter']},
   'relevance': 0.884239,
   'sentiment': {'label': 'positive', 'score': 0.128961},
   'text': 'Bill Cosby',
   'type': 'Person'},
  {'count': 2,
   'disambiguation': {'dbpedia_resource': 'http://dbpedia.org/resource/The_Cosby_Show',
    'name': 'The Cosby Show',
    'subtype': ['AwardWinner']},
   'relevance': 0.305668,
   'sentiment': {'label': 'positive', 'score': 0.62461},
   'text': 'The Cosby Show',
   'type': 'TelevisionShow'},
  {'count': 2,
   'relevance': 0.295776,
   'sentiment': {'label': 'neutral

In [4]:
def extractEntities(url, source):
    response = query_api(url)

    results = []
    
    if "entities" not in response:
        return results
    else:
        entities = response["entities"]
    
    for entity in entities:
        if ('disambiguation' not in entity) or ('name' not in entity['disambiguation']):
            continue
            
        disambiguated = entity.get('disambiguation').get('name')
        result = {
           'entity'    : disambiguated,
           'source'    : source,
           'url'       : url,
           'relevance' : entity.get('relevance'),
           'sentiment' : entity.get('sentiment').get('score')
        }
        results.append(result)

    return results


In [5]:
url = 'https://apnews.com/c42aa22bb2f04de79d192b7cbd0c3fe9'
extractEntities(url, 'Bloomberg')

[{'entity': 'Bill Cosby',
  'relevance': 0.884239,
  'sentiment': 0.128961,
  'source': 'Bloomberg',
  'url': 'https://apnews.com/c42aa22bb2f04de79d192b7cbd0c3fe9'},
 {'entity': 'The Cosby Show',
  'relevance': 0.305668,
  'sentiment': 0.62461,
  'source': 'Bloomberg',
  'url': 'https://apnews.com/c42aa22bb2f04de79d192b7cbd0c3fe9'},
 {'entity': 'Norristown, Pennsylvania',
  'relevance': 0.18949,
  'sentiment': -0.373384,
  'source': 'Bloomberg',
  'url': 'https://apnews.com/c42aa22bb2f04de79d192b7cbd0c3fe9'},
 {'entity': 'Kevin Steele',
  'relevance': 0.13324,
  'sentiment': -0.597223,
  'source': 'Bloomberg',
  'url': 'https://apnews.com/c42aa22bb2f04de79d192b7cbd0c3fe9'},
 {'entity': 'Montgomery County, Pennsylvania',
  'relevance': 0.124645,
  'sentiment': 0.0,
  'source': 'Bloomberg',
  'url': 'https://apnews.com/c42aa22bb2f04de79d192b7cbd0c3fe9'},
 {'entity': "Stephen O'Neill",
  'relevance': 0.123224,
  'sentiment': 0.556912,
  'source': 'Bloomberg',
  'url': 'https://apnews.com/

In [6]:
import pandas as pd
entities = extractEntities(url, 'Bloomberg')
df = pd.DataFrame(entities)
df

,entity,relevance,sentiment,source,url
0,Bill Cosby,0.884239,0.128961,Bloomberg,https://apnews.com/c42aa22bb2f04de79d192b7cbd0...
1,The Cosby Show,0.305668,0.624610,Bloomberg,https://apnews.com/c42aa22bb2f04de79d192b7cbd0...
2,"Norristown, Pennsylvania",0.189490,-0.373384,Bloomberg,https://apnews.com/c42aa22bb2f04de79d192b7cbd0...
3,Kevin Steele,0.133240,-0.597223,Bloomberg,https://apnews.com/c42aa22bb2f04de79d192b7cbd0...
4,"Montgomery County, Pennsylvania",0.124645,0.000000,Bloomberg,https://apnews.com/c42aa22bb2f04de79d192b7cbd0...
5,Stephen O'Neill,0.123224,0.556912,Bloomberg,https://apnews.com/c42aa22bb2f04de79d192b7cbd0...


Part 3

In [7]:
def get_entity_discussions(sources):
    results = []
    for src in sources:
        for url in getNews(src):
            entities = extractEntities(url, src)
            results.extend(entities)
    return results


In [8]:
import pandas as pd

sources = ['The-Wall-Street-Journal', 'The-New-York-Times']
entities = get_entity_discussions(sources)
df = pd.DataFrame(entities)
df

,entity,relevance,sentiment,source,url
0,Lisa Murkowski,0.802847,-0.424042,The-Wall-Street-Journal,https://www.wsj.com/articles/kavanaugh-vote-to...
1,Brett Kavanaugh,0.735802,0.000000,The-Wall-Street-Journal,https://www.wsj.com/articles/kavanaugh-vote-to...
2,Mitch McConnell,0.625436,0.000000,The-Wall-Street-Journal,https://www.wsj.com/articles/kavanaugh-vote-to...
3,Federal Bureau of Investigation,0.599096,0.000000,The-Wall-Street-Journal,https://www.wsj.com/articles/kavanaugh-vote-to...
4,United States Senate,0.456214,0.000000,The-Wall-Street-Journal,https://www.wsj.com/articles/kavanaugh-vote-to...
5,Alaska,0.437571,0.000000,The-Wall-Street-Journal,https://www.wsj.com/articles/kavanaugh-vote-to...
6,HSBC,0.962661,0.000000,The-Wall-Street-Journal,https://www.wsj.com/articles/the-500-million-c...
7,Richard Neal,0.735617,0.000000,The-Wall-Street-Journal,https://www.wsj.com/articles/trumps-tax-return...
8,United States Senate,0.499204,0.000000,The-Wall-Street-Journal,https://www.wsj.com/articles/trumps-tax-return...
9,Massachusetts,0.259434,0.000000,The-Wall-Street-Journal,https://www.wsj.com/articles/trumps-tax-return...
